- Any live cell with two or three live neighbors lives on to the next generation.
- Any empty cell with exactly three live neighbors becomes a live cell in the next generation.
- All other cells are empty in the next generation.

https://github.com/norvig/pytudes/blob/master/ipynb/Life.ipynb

### TODOs
1. Use Jack to gen same pattern, instead of python code to gen.
2. Define JACK program API interfaces: draw(cell), empty(cell)

In [1]:
import time
from IPython.display import clear_output, display_html

LIVE  = '@'
EMPTY = '.'
PAD   = ' '

HEIGHT = 10
LENGTH = 10

def neighbors(cell):
    "All 8 adjacent neighbors of cell."
    (x, y) = cell
    return [(x-1, y-1), (x, y-1), (x+1, y-1), 
            (x-1, y),             (x+1, y), 
            (x-1, y+1), (x, y+1), (x+1, y+1)]

def count_lives(cell, world):
    nbs = neighbors(cell)
    l = [n for n in nbs if n in set(world)]
    return len(l)

def find_empty_cells(world):
    cells = [(x, y) for x in range(HEIGHT) for y in range(LENGTH)]
    #print(len(cells))
    empty_cells = [c for c in cells if c not in set(world)]
    #print(len(empty_cells))
    return empty_cells

def find_new_lives(empty_cells, world):
    lives = [c for c in empty_cells if count_lives(c, world) == 3]
    return lives
    

def display(world):
    xs = range(HEIGHT)
    ys = range(LENGTH)
    board = []
    for y in ys:
        row = PAD.join([LIVE if (x, y) in world else EMPTY for x in xs])
        board.append(row)

    board = '\n'.join(board)
    return board

def next_gen(world):
    new_world = []
    for cell in world:
        c = count_lives(cell, world)
        if c == 3 or c == 2:
            new_world.append(cell)
        
    empty_cells = find_empty_cells(world)
    lives = find_new_lives(empty_cells, world)
    new_world.extend(lives)
    
    return new_world


world1 = [(1, 1), (2, 2), (3, 3), (2, 3)]
world = [ (3, 4), (3, 5), (3, 6)] # Blinker

def run(world):
    for _ in range(50):
        clear_output(wait=True)
        print(display(world))
        world = next_gen(world)
        time.sleep(0.5)

world2 = [(3, 3), (4, 3), (3, 4), (4, 4), 
          (5, 5), (6, 5), (5, 6), (6, 6)]  # Beacon
 
run(world2)

. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .
. . . @ @ . . . . .
. . . @ . . . . . .
. . . . . . @ . . .
. . . . . @ @ . . .
. . . . . . . . . .
. . . . . . . . . .
. . . . . . . . . .


In [21]:
# Quick map
xs=5
ys=5
for x in range(xs):
    l = ['.' for y in range(ys)]
    print(' '.join(l))

. . . . .
. . . . .
. . . . .
. . . . .
. . . . .


# python to Jack bitmap

In [3]:
xs=32
ys=16
for y in range(ys):
    l = ['.' for x in range(xs)]
    print(' '.join(l))

. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . 

In [12]:

addr = 1
for i in range(16):
    code = 'do Memory.poke(memAddress+%d, -1);' % addr
    print(code)
    addr += 32
    

do Memory.poke(memAddress+1, -1);
do Memory.poke(memAddress+33, -1);
do Memory.poke(memAddress+65, -1);
do Memory.poke(memAddress+97, -1);
do Memory.poke(memAddress+129, -1);
do Memory.poke(memAddress+161, -1);
do Memory.poke(memAddress+193, -1);
do Memory.poke(memAddress+225, -1);
do Memory.poke(memAddress+257, -1);
do Memory.poke(memAddress+289, -1);
do Memory.poke(memAddress+321, -1);
do Memory.poke(memAddress+353, -1);
do Memory.poke(memAddress+385, -1);
do Memory.poke(memAddress+417, -1);
do Memory.poke(memAddress+449, -1);
do Memory.poke(memAddress+481, -1);


In [26]:
def _find_start_pos_in_mem(r, c):
#     We re-map the 512 * 256 Screen to a smaller board: 32 * 16 (col * row).
#     Column (0 <= r <= 31). Row(0 <= row <= 16)
#     Each row has 32 words, each word can either be filled or empty.

#     In our 32 * 16 board, each cell is a 16 * 16 bits square.
#     The size of cell in Screen is 32 * 16, where 32 is Screen's row size.

#     Example: To display cell (1, 1) from new board in Screen:
#     Row is 'row * 32 * 16', and column is just 'col'. 
#     The mem address is: row * 32 * 16 + col.
    return c * 32 * 16 + r

def gen_jack_screen_code_from_cell(cell):
    x, y = cell
    addr = _find_start_pos_in_mem(x, y)
    for i in range(16):
        code = 'do Memory.poke(memAddress+%d, -1);' % addr
        print(code)
        addr += 32

#gen_jack_screen_code_from_cell((1, 0))
#gen_jack_screen_code_from_cell((2, 1))
#gen_jack_screen_code_from_cell((3, 2))
#gen_jack_screen_code_from_cell((4, 3))
gen_jack_screen_code_from_cell((31, 15))

do Memory.poke(memAddress+7711, -1);
do Memory.poke(memAddress+7743, -1);
do Memory.poke(memAddress+7775, -1);
do Memory.poke(memAddress+7807, -1);
do Memory.poke(memAddress+7839, -1);
do Memory.poke(memAddress+7871, -1);
do Memory.poke(memAddress+7903, -1);
do Memory.poke(memAddress+7935, -1);
do Memory.poke(memAddress+7967, -1);
do Memory.poke(memAddress+7999, -1);
do Memory.poke(memAddress+8031, -1);
do Memory.poke(memAddress+8063, -1);
do Memory.poke(memAddress+8095, -1);
do Memory.poke(memAddress+8127, -1);
do Memory.poke(memAddress+8159, -1);
do Memory.poke(memAddress+8191, -1);


In [82]:
import time
from IPython.display import clear_output, display_html

LIVE  = '@'
EMPTY = '.'
PAD   = ' '


class World:

  def find_index_from_cell(self, col, row):
    
    if col < 0 or col > 31:
        return -1
    
    if row < 0 or row > 15:
        return -1
    
    index = 32 * row + col
    
    return index


  def math_mod(self, c, m):
    if c >= m:
        result = c - (int(c / m) * m)
    else:
        result = c
    return result

  def _find_cell_from_index(self, index):
    x = int(self.math_mod(index, 32))
    y = int(index / 32)
    return x, y

  def is_filled(self, world, i):
    
    return world[i]
  

  def _count_as_life_if_filled(self, world, c, r, lives):

    index = self.find_index_from_cell(c, r)
    if index == -1:
        return lives
    
    if self.is_filled(world, index):
      lives = lives + 1
    
    return lives

  def count_neighbors_lives_from_cell(self, world, col, row):
    lives = 0
    x = col
    y = row
      
    c = x-1
    r = y-1
    lives = self._count_as_life_if_filled(world, c, r, lives)

    c = x
    r = y-1
    lives = self._count_as_life_if_filled(world, c, r, lives)

    c = x+1
    r = y-1
    lives = self._count_as_life_if_filled(world, c, r, lives)

    c = x-1
    r = y
    lives = self._count_as_life_if_filled(world, c, r, lives)

    c = x+1
    r = y
    lives = self._count_as_life_if_filled(world, c, r, lives)

    c = x-1
    r = y+1
    lives = self._count_as_life_if_filled(world, c, r, lives)

    c = x
    r = y+1
    lives = self._count_as_life_if_filled(world, c, r, lives)

    c = x+1
    r = y+1
    lives = self._count_as_life_if_filled(world, c, r, lives)

    return lives
  
    
  def display(self, world):
    xs = range(32)
    ys = range(16)
    board = []
    for y in ys:
        l = []
        for x in xs:
            index = self.find_index_from_cell(x, y)
            
            if world[index] == 1:
                l.append(LIVE)
            else:
                l.append(EMPTY)

        row = PAD.join(l)
                
        board.append(row)

    board = '\n'.join(board)
    return board

def run():
    w = World()
    cur_world = [0] * 512
    cur_world[2] = 1
    cur_world[34] = 1
    cur_world[66] = 1
    
    cur_world[110] = 1
    cur_world[111] = 1
    cur_world[142] = 1
    cur_world[143] = 1
    
    cur_world[176] = 1
    cur_world[177] = 1
    cur_world[208] = 1
    cur_world[209] = 1

    count = 0
    while (count < 50):
      clear_output(wait=True)

      print(w.display(cur_world))
      
      # pause
      time.sleep(0.5)

      new_world = [0] * 512
      index = 0
      while (index < 512):
        x, y = w._find_cell_from_index(index);
        #print(index, x, y)
        lives = w.count_neighbors_lives_from_cell(cur_world, x, y);
        if lives > 0:
            msg = "x=%d,y=%d, lives=%d, index=%d" % (x, y, lives, index)
            #print(msg)
        new_world[index] = 0;
        if w.is_filled(cur_world, index):
          if lives == 3 or lives == 2:
            new_world[index] = 1

        elif lives == 3:
          new_world[index] = 1

        index = index + 1

      i = 0;
      while i < 512:
        if new_world[i] == 1:
          cur_world[i] = 1
        else:
          cur_world[i] = 0

        i = i + 1

      count = count + 1

run()

def test(i):
    w = World()
    x, y = w._find_cell_from_index(i)
    print("index", x, y)
    
#test(2)



. . @ . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . @ . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . @ . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . @ @ . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . @ @ . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . @ @ . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . @ @ . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . 

KeyboardInterrupt: 